# Requirements

- Putting src data from BQ, into the workspace bucket
- id_rsa key and placement in the ~ directory in Terra
- Dirs in 

# Process

- Run this notebook from the Terra Analysis page. The copy stored in the repository is kept for source control.
- Don't run all the cells at once without reviewing them, parameters toward the bottom, will need to be updated first.


import separate files to bucket via BQ 
then run pipeline_helpers to get them into /data
<!-- then run the import macro to get them into duckdb -->
duckdb can pull from a path. -- can skip import if they exist in data dir.

dbt run-operation register_external_sources --args '{"fq_tablename":"sample_ANVIL_CMG_YALE_DS_MC_20221026_ANV5_202409302315_000000000000","src_loc":"data/cmg_yale/sample_ANVI

# Review/Edit all of the following parameters

In [ ]:
pipeline = 'anvil_dbt_project'
repo='git@github.com:NIH-NCPI/anvil_dbt_project.git' #The ssh version
study_id = 'cmg_yale'
org_id = 'anvil'
dbt_repo = 'anvil_dbt_project'
tgt_model_id = 'tgt_consensus_a'

ftd_schema = f'main_{study_id}_data'
tgt_schema = f'main_{study_id}_tgt_data'

gh_email = ''
gh_user = ''

# NB and dbt setup

In [ ]:
# Imports
from pathlib import Path
import os
import pandas as pd
import duckdb
from jinja2 import Template
import sys

bucket = os.environ['WORKSPACE_BUCKET']
con = duckdb.connect("/tmp/dbt.duckdb")

project_root = Path().resolve().parent
sys.path.append(str(project_root))
from scripts.general import get_all_paths as get_all_paths
paths = get_all_paths(study_id, dbt_repo, org_id, tgt_model_id, src_data_path=None)

# print(f'INFO: Complete {repo_dir}')

In [ ]:
paths

In [ ]:
def create_file_dict(table, count):
    file_list = []
    for i in range(count):
        if i == 0:
            file = f'{table}_{"0" * 12}.csv'
        else:
            file = f'{table}_{"0" * (12 - len(str(i)))}{i}.csv'
        file_list.append(file)
    
    return {table: file_list}

def get_bucket_src_data_format_store(src_table_list):
    '''
    Data files are a special case. Get them from the bucket with this 
    function, NOT pull_study_files()
    Data files should not be edited manually. If edits are required, 
    use the original queries with edits to store the new data in the bucket
    '''
    print('INFO: Start')

    copy_data_from_bucket(paths['bucket_study_dir'], src_files, paths['src_data_dir'])

    # Iterate over the dictionaries and process files
    for file_dict in partial_file_dicts:  # Iterate over each dictionary in partial_file_dicts
        for table, file_list in file_dict.items():  # Extract table name (key) and file list (value)
            # Concatenate files for the current table
            read_and_concat_files(file_list, paths['src_data_dir'], f'{paths['src_data_dir']}/{table}_combined.csv')

    # Rename the concatenated files
    for table in src_table_list:  # Iterate over all table names
        rename_file_single_dir(paths['src_data_dir'], f'{table}_combined.csv', f'{table}.csv')

    # Remove all the files in the dictionaries
    for file_dict in partial_file_dicts:  # Iterate over each dictionary in partial_file_dicts
        for table, file_list in file_dict.items():
            remove_file(file_list, paths['src_data_dir'])

    print('INFO: Complete')
    
def setup_ssh():
    # Create and configure ~/.ssh
    if not paths['ssh_dir'].is_dir():
        paths['ssh_dir'].mkdir(mode=0o700, exist_ok=True)
        print("INFO: Created ~/.ssh directory.")
    ssh_config = paths['ssh_dir'] / "config"
    if not ssh_config.exists():
        ssh_config.write_text(
            """# SSH configuration for GitHub
Host github
  HostName github.com
  User git
  IdentityFile ~/.ssh/id_rsa
  IdentitiesOnly yes
"""
        )
        ssh_config.chmod(0o600)
        print("INFO: Created ~/.ssh/config file.")
        
 # Move id_rsa to ~/.ssh and set correct permissions
    if id_rsa_src.exists():
        os.system(f"mv {id_rsa_src} {id_rsa_dest}")
        id_rsa_dest.chmod(0o600)
        print(f"INFO: Moved id_rsa to {id_rsa_dest} and set permissions to 600.")

    if not id_rsa_src.exists() and not id_rsa_dest.exists():
        print(f"WARNING: Make sure the private key is available.")

# See [docs](https://github.com/DataBiosphere/terra-examples/blob/main/best_practices/source_control/terra_source_control_cheatsheet.md#1-use-the-jupyter-console-to-upload-your-github-ssh-key-and-create-an-interactive-terminal-session) 
def setup_gh():
    content1=f'''
[user]
        email = {gh_email}
        name = {gh_user}
[url "git@github.com:"]
        insteadOf = https://github.com/

    '''

    with git_config_path.open("a") as file:
        file.write("\n" + content1)

    print("INFO: Edited ~/.gitconfig file.")
        
def update_bash_profile():
    
    content1 = """
# Custom PS1 prompt with virtual environment display
export PS1='\\[\\033[1;33m\\]${VIRTUAL_ENV:+(venv)} \\[\\033[1;36m\\]$(basename "$PWD")\\[\\033[00m\\]\\$ '
"""

    content2 = f"""
# Add SSH private key
eval "ssh-add ~/.ssh/id_rsa"

# Alias to activate Python virtual environment
alias activate="source /home/jupyter/venv-python3.12/bin/activate"

# Alias to setup ssh and permissions:
alias setup_ssh="
echo 'Assuming the id_rsa is in the {paths['id_rsa_src']} dir'
eval 'mv {paths['id_rsa_src']} {paths['id_rsa_dest']}'
eval 'chmod 600 ~/.ssh/id_rsa'
"

# Setup dirs and clone the repo
alias clone_repo="
eval 'mkdir {paths['pipeline_dir']}'
eval 'mkdir {paths['repo_home_dir']}'
eval 'git clone {repo} {paths['repo_home_dir']}'
eval 'mkdir paths['src_data_dir']'
eval 'activate'
eval 'cd {paths['repo_home_dir']}'
eval 'mkdir {paths['output_dir']}'
eval 'mkdir {paths['outputz_study_dir']}'
"

# Alias to dbt prep file system:
alias setup_data="
eval 'mkdir {paths['dbt_dir']}'
eval 'cp {paths['profiles_path_root']} {paths['profiles_path_home']}'
"

# Alias to clean and compile pipeline:
alias r_dbt="
eval 'dbt clean'
eval 'dbt deps'
"

echo 'Alias are: activate, r_dbt, setup_ssh clone_repo, setup_data'

export LOCUTUS_LOGLEVEL='INFO'

"""
    with bash_profile.open("a") as file:
        file.write("\n" + content1 + "\n" + content2)
        
    print("INFO: Content successfully added to ~/.bash_profile.")

    print("INFO: To apply changes, run: source ~/.bash_profile")

def stop_gitignoring_sql_files():
    content = """
!*.sql
!*.yml
"""
    with paths['terra_gitignore'].open("a") as file:
        file.write("\n" + content + "\n")
    print("INFO: Content successfully added to ~/gitignore_global")
    
    
def run_initial_setup():
    '''
    Run the setup functions
    '''
    setup_ssh() # Required first time env setup
    setup_gh() # Required first time env setup
    update_bash_profile()
    stop_gitignoring_sql_files()
def copy_data_from_bucket(bucket_study_dir, file_list, output_dir):
    for file in file_list:
#         TODO: checkout rsync https://google-cloud-how-to.smarthive.io/buckets/rsync
        !gsutil cp {bucket_study_dir}/{file} {output_dir}
        print(f'INFO: Copied {file} to {output_dir}') 

def copy_data_to_bucket(bucket_study_dir, file_list, input_dir):
    for file in file_list:
        !gsutil cp {input_dir} {bucket_study_dir}/{file}
        print(f'INFO: Copied {file} to the bucket') 
        
# Read and concatenate all files
def read_and_concat_files(file_list, input_dir, output_dir):
    dfs = [pd.read_csv(f'{input_dir}/{file}') for file in file_list] 
    combined_subject = pd.concat(dfs, ignore_index=True)
    combined_subject.to_csv(output_dir, index=False)
    
def rename_file_single_dir(d_dir, input_fn, output_fn):
    # clean up data_dir
    !mv {d_dir}/{input_fn} {d_dir}/{output_fn}
    
def remove_file(file_list, d_dir):
    for file in file_list:
        !rm {d_dir}/{file}
        print(f'INFO: Processsed: {file}')
    
    # Save the files before shutting down an environment
def store_study_files():
    """ Store defined files in the bucket. These will persist when env is shutdown."""
    for file in study_files:
        !gsutil cp {paths['src_data_dir']}/{file} {paths['bucket_study_dir']}
    for file in seeds_files:
        !gsutil cp {paths['seeds_dir']}/{file} {paths['bucket_study_dir']}
        
def get_study_files():
    """ Pull study files from where they are saved in the workspace bucket.
    SHOULD NOT get datafiles. Run get_bucket_src_data_format_store for those."""
    for file in study_files:
        !gsutil cp  {paths['bucket_study_dir']}/{file} {paths['src_data_dir']}/{file}
    for file in seeds_files:
        !gsutil cp  {paths['bucket_study_dir']}/{file} {paths['seeds_dir']}/{file}
    for file in other_files:
        !gsutil cp  {paths['bucket']}/{file} {paths['src_data_dir']}/{file}
        
    
# Export functions       
def get_tables_from_schema(schema):
    '''
    Get tables from a duckdb dataset. 
    '''
    result = con.execute(f"""
    SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema}'
    """)
    r = pd.DataFrame(result.fetchall(), columns=[col[0] for col in result.description])
    return r['table_name'].to_list()

def tables_to_output_dir(tables):
    for t in tables:
        name = Path(t).stem.replace(f'tgt_','')
        t = con.execute( f"COPY (SELECT * FROM {tgt_schema}.{t}) TO '{paths['output_study_dir']}/{name}.csv' (HEADER, DELIMITER ',')").fetchall()
        print(name)

def harmonized_to_bucket(tables):
    for t in tables:
        name = Path(t).stem.replace(f'tgt_','')
        !gsutil cp {paths['output_study_dir']}/{name}.csv {paths['bucket']}/harmonized/{study_id}
        print(name)


def copy_to_csv_and_export_to_bucket():    
    '''
    Get the tables that you want to export to csv.
    Then export to csv in the output dir
    '''
    tgt_tables = get_tables_from_schema(tgt_schema)

    tables_to_output_dir(tgt_tables)
    display('Tables sent to output.')
    
    harmonized_to_bucket(tgt_tables)
    display('csvs sent to bucket')
    
def convert_csv_to_utf8(input_file_path, output_filepath, delimiter, encoding):
    df = pd.read_csv(input_file_path, encoding=encoding, delimiter=delimiter, quoting=3)
    df.to_csv(output_filepath, index=False, encoding='utf-8')
    print(f"Converted CSV saved to {output_filepath}")

# Study specific parameters
- Add another if starting a new study

In [ ]:
"""  
Template for adding a new study
"""
if study_id == '______':
    src_table_list = ['______','______']

    # files needing concat
    subject_pfiles = create_file_dict('______', ______)
    sample_pfiles = create_file_dict('______', ______)
    
    partial_file_dicts = [ _______pfiles, _______pfiles]
    src_files = [file for file_dict in partial_file_dicts for file_list in file_dict.values() for file in file_list]
    src_table_list = [key for file_dict in partial_file_dicts for key in file_dict.keys()]

    # Define the files to store
    study_files = ['_______dd.csv',
                   '_______dd.csv'
                  ]

    seeds_files = [] # Insert seed files if applicable


In [ ]:
"""  CMG-BH specific
Only edit if working on cmg_bh
"""
if study_id == 'cmg_bh':
    src_table_list = ['subject','sample']

    # files needing concat
    subject_pfiles = create_file_dict('subject', 2)
    sample_pfiles = create_file_dict('sample', 1)
    
    partial_file_dicts = [subject_pfiles, sample_pfiles]
    src_files = [file for file_dict in partial_file_dicts for file_list in file_dict.values() for file in file_list]
    src_table_list = [key for file_dict in partial_file_dicts for key in file_dict.keys()]

    # Define the files to store
    study_files = ['subject_dd.csv',
                   'sample_dd.csv'
                  ]

    seeds_files = ['cmg_bh_annotations_code.csv', 'subject_mappings.csv']
    other_files = ['terra_startup_script.sh']


In [ ]:
"""  CMG-Yale specific
Only edit if working on cmg_yale
"""
if study_id == 'cmg_yale':
    src_table_list = [
                      'subject'
                      ,'sample'
                      ,'anvil_dataset'
                      ,'sequencing'
                      ,'family'
                      ,'file_inventory'
                     ]
    
    tables = ['anvil_dataset']

    datasets = ['datarepo-c2a2b724.ANVIL_CMG_YALE_DS_MC_20221026_ANV5_202409302315',
               'datarepo-5c7f0d2a.ANVIL_CMG_Yale_HMB_IRB_20240113_ANV5_202410011846',
               'datarepo-fe056168.ANVIL_CMG_Yale_DS_RD_20240113_ANV5_202410011804',
               'datarepo-f961f617.ANVIL_CMG_YALE_DS_RARED_20221020_ANV5_202409251714',
               'datarepo-06182245.ANVIL_CMG_Yale_DS_THAL_IRB_20240113_ANV5_202410011814',
               'datarepo-35779fe0.ANVIL_CMG_Yale_HMB_20221020_ANV5_202410011825',
               'datarepo-cebe6de0.ANVIL_CMG_Yale_HMB_GSO_20221020_ANV5_202410011834',
               'datarepo-5d222190.ANVIL_CMG_Yale_DS_BPEAKD_20240113_ANV5_202410011754',
               'datarepo-2b98851b.ANVIL_CMG_Yale_GRU_20221020_ANV5_202507091800'
              ]

    src_table_list = []
    for table in tables:
        for ds in datasets:
            tablename = ds.split('.', 1)[1]
            src_table_list.append(f"{table}_{tablename}_000000000000.csv")

    # files needing concat
#     subject_pfiles = create_file_dict('subject', 9)
#     sample_pfiles = create_file_dict('sample', 9)
#     anvil_dataset_pfiles = create_file_dict('anvil_dataset', 9) 
#     sequencing_pfiles = create_file_dict('sequencing', 4) 
#     family_pfiles = create_file_dict('family', 9) 
#     file_inventory_pfiles = create_file_dict('file_inventory', 9) 
    
#     partial_file_dicts = [subject_pfiles
#                           , sample_pfiles
#                           , anvil_dataset_pfiles
#                           , sequencing_pfiles
#                           , family_pfiles
#                           , file_inventory_pfiles
#                          ]
#     src_files = [file for file_dict in partial_file_dicts for file_list in file_dict.values() for file in file_list]
#     src_table_list = [key for file_dict in partial_file_dicts for key in file_dict.keys()]

    # Define the files to store
    study_files = ['subject_dd.csv',
                   'sample_dd.csv',
                   'family_dd.csv',
                   'sequencing_dd.csv',
                   'file_inventory_dd.csv',
                   'cmg_yale_study.yaml',
                   'ftd_study.yaml',
                   'Yale_CMG_Master_DD.csv'
                  ]
    
    seeds_files = []
    other_files = ['terra_startup_script.sh']
    nbs = ['pipeline_helpers.ipynb', 'raw_data_validation.ipynb']
#     print(src_files)

In [ ]:
"""  
Template for adding a new study
"""
if study_id == '______':
    src_files = []
    src_table_list = []
    if study_id == 'study_here':
        src_table_list = ['table1','table2']

        """
        Handles files in the bucket needing concat(pfiles - Partial files).
        Ex: ['subject_000000000001','subject_000000000002']
        Optional. Depends on how the data was imported into the workspace bucket.
        Comment them out if not needed.
        """
        subject_pfiles = create_file_dict('{table}', max_file_n_plus_one)
        sample_pfiles = create_file_dict('{table}', max_file_n_plus_one)

        partial_file_dicts = [] # List of the pfiles created above. Must have the suffix '_pfiles'. 


        # No edits necessary.
        if len(partial_file_dicts) != 0:
            src_files = [file for file_dict in partial_file_dicts for file_list in file_dict.values() for file in file_list]
            src_table_list = [key for file_dict in partial_file_dicts for key in file_dict.keys()]



        # Define the files to store in the bucket that exist in the data/{study_id} dir.
        study_files = ['_______dd.csv',
                       '_______dd.csv'
                      ]

        seeds_files = [] # Insert seed files to send to the bucket if applicable

        # Define the files to pull from the bucket(not from a study dir inside the bucket), placed in the data/{study_id} dir
        other_files = [] 

        # Define notebooks from the analysis dir that need to be updated in the GH repo
        nbs = []

# Run functions
- Enable functions and run one at a time

In [ ]:
'''
If starting a new pipeline env
- After putting your Private id_rsa key file in the home dir in Terra
1. Set up GH and terminal configurations
- Run 'run_initial_setup' in this cell
- Go to terminal and run:
    - 'source ~/.bash_profile' - A list of available commands should show up
    - 'setup_ssh'
    - 'clone_repo'
    - 'setup_data'
- At this point you should be able to connect to GitHub and swap branches
'''
# run_initial_setup()





'''
Get the dds and config files from the bucket
'''
# get_study_files() 





'''
Get the src data files from the bucket
'''
# get_bucket_src_data_format_store(src_table_list)

"""
Run the following if you only want to pull in the data files from the bucket. Run this 
if there are no 'partial' files to combine.
"""
# copy_data_from_bucket(paths['bucket_study_dir'], src_table_list, paths['src_data_dir'])






'''
Put study files into the bucket.
'''
# store_study_files()




'''
Export tgt data to csvs in the output dir. Then send the files to the harmonized dir in the bucket
'''
# copy_to_csv_and_export_to_bucket()




"""
Convert files in data dir into utf-8. Add to the appropriate list, to save the changes in the bucket.
"""
# input_filepath = f'{seeds_dir}/value_sets/RoleCodeValueSet.csv'
# output_filepath = f'{seeds_dir}/value_sets/RoleCodeValueSet.csv'
# delimiter = '\t'
# encoding = 'latin1'
# convert_csv_to_utf8(input_filepath, output_filepath, delimiter, encoding)

print('Completed')

In [ ]:
samples = ['ANVIL_CMG_YALE_DS_MC_20221026_ANV5_202409302315',
           'ANVIL_CMG_Yale_HMB_IRB_20240113_ANV5_202410011846',
           'ANVIL_CMG_Yale_DS_RD_20240113_ANV5_2024100118045',
           'ANVIL_CMG_YALE_DS_RARED_20221020_ANV5_202409251714',
           'ANVIL_CMG_Yale_DS_THAL_IRB_20240113_ANV5_202410011814',
           'ANVIL_CMG_Yale_HMB_20221020_ANV5_202410011825',
           'ANVIL_CMG_Yale_HMB_GSO_20221020_ANV5_202410011834',
           'ANVIL_CMG_Yale_DS_BPEAKD_20240113_ANV5_202410011754',
           'ANVIL_CMG_Yale_GRU_20221020_ANV5_202507091800'
          ]

for file in samples:
    print(f"""
    EXPORT DATA OPTIONS(uri='{bucket}/cmg_yale/sample_*.csv', format='CSV', overwrite=true ,header=true) AS 
    (
    SELECT *
    FROM `datarepo-5c7f0d2a.{file}.sample`
    );
    """)

In [ ]:
samples = ['ANVIL_CMG_YALE_DS_MC_20221026_ANV5_202409302315',
           'ANVIL_CMG_Yale_HMB_IRB_20240113_ANV5_202410011846',
           'ANVIL_CMG_Yale_DS_RD_20240113_ANV5_2024100118045',
           'ANVIL_CMG_YALE_DS_RARED_20221020_ANV5_202409251714',
           'ANVIL_CMG_Yale_DS_THAL_IRB_20240113_ANV5_202410011814',
           'ANVIL_CMG_Yale_HMB_20221020_ANV5_202410011825',
           'ANVIL_CMG_Yale_HMB_GSO_20221020_ANV5_202410011834',
           'ANVIL_CMG_Yale_DS_BPEAKD_20240113_ANV5_202410011754',
           'ANVIL_CMG_Yale_GRU_20221020_ANV5_202507091800'
          ]

for file in samples:
    print(f"""
    EXPORT DATA OPTIONS(uri='{bucket}/cmg_yale/sample_*.csv', format='CSV', overwrite=true ,header=true) AS 
    (
    SELECT *
    FROM `datarepo-5c7f0d2a.{file}.sample`
    );
    """)

In [ ]:
tables = ['subject','sample','family','sequencing','file_inventory']

datasets = ['datarepo-c2a2b724.ANVIL_CMG_YALE_DS_MC_20221026_ANV5_202409302315',
           'datarepo-5c7f0d2a.ANVIL_CMG_Yale_HMB_IRB_20240113_ANV5_202410011846',
           'datarepo-fe056168.ANVIL_CMG_Yale_DS_RD_20240113_ANV5_202410011804',
           'datarepo-f961f617.ANVIL_CMG_YALE_DS_RARED_20221020_ANV5_202409251714',
           'datarepo-06182245.ANVIL_CMG_Yale_DS_THAL_IRB_20240113_ANV5_202410011814',
           'datarepo-35779fe0.ANVIL_CMG_Yale_HMB_20221020_ANV5_202410011825',
           'datarepo-cebe6de0.ANVIL_CMG_Yale_HMB_GSO_20221020_ANV5_202410011834',
           'datarepo-5d222190.ANVIL_CMG_Yale_DS_BPEAKD_20240113_ANV5_202410011754',
           'datarepo-2b98851b.ANVIL_CMG_Yale_GRU_20221020_ANV5_202507091800'
          ]

for table in tables:
    for ds in datasets:
        tablename = ds.split('.', 1)[1]
        print(f"""
        EXPORT DATA OPTIONS(uri='{bucket}/{study_id}/{table}_{tablename}_*.csv', format='CSV', overwrite=true ,header=true) AS 
        (
        SELECT *
        FROM `{ds}.{table}`
        );
        """)



In [ ]:
for ds in datasets:
    tablename = ds.split('.', 1)[1]

    print(f"""
    EXPORT DATA OPTIONS(uri='{bucket}/cmg_yale/anvil_dataset_{tablename}_*.csv', format='CSV', overwrite=true ,header=true) AS 
    (
      SELECT 
      dataset_id,
      ARRAY_TO_STRING(consent_group, ',') AS consent_group,
      ARRAY_TO_STRING(data_use_permission, ',') AS data_use_permission,
      ARRAY_TO_STRING(owner, ',') AS owner,
      ARRAY_TO_STRING(principal_investigator, ',') AS principal_investigator,
      ARRAY_TO_STRING(registered_identifier, ',') AS registered_identifier,
      title,
      ARRAY_TO_STRING(data_modality, ',') AS data_modality,
      ARRAY_TO_STRING(source_datarepo_row_ids, ',') AS source_datarepo_row_ids
    FROM `{ds}.anvil_dataset`
        );
        """)

In [ ]:
for table in tables:
    for ds in datasets:
        tablename = ds.split('.', 1)[1]
        print(f"""
        dbt run-operation register_external_sources --args '{{"fq_tablename": "{table}_{tablename}_000000000000", "src_loc": "data/{study_id}/{table}_{tablename}_000000000000.csv", "src_format": "csv"}}';
        """)